In [1]:
DATA_type = 'mels_kan_1'
sample_rate = 2000
n_fft, n_hop_length = 512, 512
window_size, hop_length = int(sample_rate*1.0), sample_rate/4
n_mels = 256
seed = 958

In [2]:
import glob
import random
import librosa
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from imblearn.under_sampling import RandomUnderSampler

In [3]:
heart_sound_all = glob.glob('../../heart-new/training/*.wav')
heart_sound, heart_sound_test = heart_sound_all[:2620], heart_sound_all[2620:]
print(np.shape(heart_sound),np.shape(heart_sound_test))

(2620,) (620,)


In [4]:
with open('../../heart-new/label-all/RECORDS') as file:Record=file.read().splitlines()

def process_labels(folder):
    with open(f'../../heart-new/{folder}/RECORDS') as file:
        records = file.read().splitlines()
    with open(f'../../heart-new/{folder}/RECORDS-normal') as file:
        normal = set(file.read().splitlines())
    with open(f'../../heart-new/{folder}/RECORDS-abnormal') as file:
        abnormal = set(file.read().splitlines())
    normal_dict = {item: 0 for item in list(normal)}
    abnormal_dict = {item: 1 for item in list(abnormal)}
    combined_dict = {**normal_dict, **abnormal_dict}
    reordered_dict = {k: combined_dict[k] for k in records}
    label_list = list(reordered_dict.values())
    return label_list

LABEL_list_all = process_labels('label-all')
LABEL_list, LABEL_list_test = LABEL_list_all[:2620], LABEL_list_all[2620:]
LABEL_array, LABEL_array_test = np.array(LABEL_list), np.array(LABEL_list_test)
print(np.shape(LABEL_array), np.shape(LABEL_array_test))

(2620,) (620,)


In [5]:
def load_data(heart_sounds, sr=sample_rate):
    data = []
    for i in tqdm(range(len(heart_sounds)),desc='Import data',leave=False):
        y, sr = librosa.load(heart_sounds[i], sr=sr)
        if len(y) >= sample_rate*5 : data.append(y[:sample_rate*5])
        else:data.append(y)
    return data
    
def augment_data(signals, labels, sr=sample_rate, fraction=1):
    num_to_augment = int(len(signals) * fraction)
    indices_to_augment = random.Random(seed).sample(range(len(signals)), num_to_augment)
    aug_signals,aug_labels = [],[]
    methods = ['pitch_shift', 'time_stretch', 'add_noise']
    for idx in tqdm(indices_to_augment,desc='Augmenting',total=num_to_augment,leave=False):
        signal = signals[idx]
        label = labels[idx]
        method = random.Random(seed).choice(methods)
        if method == 'pitch_shift':
            n_semitones = random.Random(seed).uniform(-2, 2)
            augmented_signal = librosa.effects.pitch_shift(y=signal, sr=sample_rate, n_steps=n_semitones)
        elif method == 'time_stretch':
            rate = random.Random(seed).uniform(0.8, 1.2)
            augmented_signal = librosa.effects.time_stretch(signal, rate=rate)
        elif method == 'add_noise':
            noise_factor = random.Random(seed).uniform(0.001, 0.01)
            noise = noise_factor * np.random.Random(seed).randn(len(signal))
            augmented_signal = signal + noise
        aug_signals.append(augmented_signal)
        aug_labels.append(label)
    return signals, list(labels), aug_signals, list(aug_labels)

def sliding_window(signals, labels, window_size=window_size, hop_length=hop_length):
    segments = []
    segment_labels = []
    for signal, label in tqdm(zip(signals, labels),desc='Segmenting',total=len(labels),leave=False):
        if len(signal) < window_size:
            continue
        num_segments = int((len(signal)-hop_length)/(window_size-hop_length))
        for i in range(num_segments):
            start = int(i * (window_size-hop_length))
            end = start + window_size
            window = signal[start:end]
            segments.append(window)
            segment_labels.append(label)
    return segments, segment_labels

def double_shuffle(signals, labels):
    Z = list(zip(signals, labels))
    random.Random(seed).shuffle(Z)
    shuffled_data, shuffled_label = zip(*Z)
    shuffled_data, shuffled_label = np.array(shuffled_data), np.array(shuffled_label)
    return shuffled_data, shuffled_label

def balance_classes(signals, labels):
    signals = np.array(signals, dtype=np.float16)
    labels = np.array(labels, dtype=np.int16)
    undersampler = RandomUnderSampler(random_state=seed)
    signals_resampled, labels_resampled = undersampler.fit_resample(signals, labels)
    return signals_resampled, labels_resampled

def extract_features(signals):
    mel_spectrograms = []
    scaler = MinMaxScaler()
    for signal in tqdm(signals,desc='Extracting features',total=len(signals),leave=False):
        mel_spectrogram = librosa.feature.melspectrogram(y=signal, sr=sample_rate, n_fft=n_fft, hop_length=n_hop_length, n_mels=n_mels)
        mel_spectrogram = librosa.amplitude_to_db(mel_spectrogram, ref=np.max)
        mel_spectrogram_flat = mel_spectrogram.flatten().reshape(-1,1)
        mel_spectrogram_normalized = scaler.fit_transform(mel_spectrogram_flat)
        mel_spectrogram_normalized = mel_spectrogram_normalized.reshape(mel_spectrogram.shape)
        mel_spectrograms.append(mel_spectrogram_normalized)
    return np.array(mel_spectrograms, dtype=np.float32)

def preprocessing(signals, labels, length=-1):
    signals_windowed, labels_windowed = sliding_window(signals, labels)
    signals_balanced, labels_balanced = balance_classes(signals_windowed, labels_windowed)
    signals_shuffled, labels_shuffled = double_shuffle(signals_balanced, labels_balanced)
    signals_reshaped, labels_reshaped = signals_shuffled[:length], labels_shuffled[:length]
    signal = extract_features(signals_reshaped)
    label = list(labels_reshaped)
    return signal, label

In [6]:
print("Preprocessing train & validation dataset...")
DATA_array = load_data(heart_sound)
Training_data, Training_label, augmented_data, augmented_label = augment_data(DATA_array, LABEL_array)
Training_data_preprocessed, Training_label_preprocessed = preprocessing(Training_data, Training_label)
augmented_data_preprocessed, augmented_label_preprocessed = preprocessing(augmented_data, augmented_label, length=9000-np.shape(Training_label_preprocessed)[0])
DATA = np.concatenate([Training_data_preprocessed, augmented_data_preprocessed], axis=0)
LABEL = np.array(Training_label_preprocessed + augmented_label_preprocessed, dtype=np.int32)

k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=seed)
folds = list(kf.split(DATA))

EVERY_Fold_data_train = [[] for i in range(k)]
EVERY_Fold_data_val = [[] for i in range(k)]
EVERY_Fold_label_train = [[] for i in range(k)]
EVERY_Fold_label_val = [[] for i in range(k)]

for i, (train_idx, val_idx) in enumerate(folds):
    EVERY_Fold_data_train[i].append(DATA[train_idx])
    EVERY_Fold_label_train[i].append(LABEL[train_idx])
    EVERY_Fold_data_val[i].append(DATA[val_idx])
    EVERY_Fold_label_val[i].append(LABEL[val_idx])

print("Preprocessing and fold assignment complete!")

Preprocessing train & validation dataset...


Preprocessing and fold assignment complete!


In [7]:
print("Preprocessing test dataset...")
DATA_array_test = load_data(heart_sound_test)
Test_data, Test_label = preprocessing(DATA_array_test, LABEL_array_test, length=1000)
print("Preprocessing complete!")

Preprocessing test dataset...


Preprocessing complete!


In [8]:
EVERY_Fold_data_train_download=[[],[],[],[],[]]
EVERY_Fold_label_train_download=[[],[],[],[],[]]
EVERY_Fold_data_val_download=[[],[],[],[],[]]
EVERY_Fold_label_val_download=[[],[],[],[],[]]
for i in range(5):EVERY_Fold_data_train_download[i] = EVERY_Fold_data_train[i][0]
for i in range(5):EVERY_Fold_label_train_download[i] = EVERY_Fold_label_train[i][0]
for i in range(5):EVERY_Fold_data_val_download[i] = EVERY_Fold_data_val[i][0]
for i in range(5):EVERY_Fold_label_val_download[i] = EVERY_Fold_label_val[i][0]
print(np.shape(EVERY_Fold_data_train_download))
print(np.shape(EVERY_Fold_label_train_download))
print(np.shape(EVERY_Fold_data_val_download))
print(np.shape(EVERY_Fold_label_val_download))
print(np.shape(Test_data))
print(np.shape(Test_label))

(5, 7200, 256, 4)
(5, 7200)
(5, 1800, 256, 4)
(5, 1800)
(1000, 256, 4)
(1000,)


In [9]:
train_fraction , val_fraction = len(LABEL)*(k-1)/(100*k) , len(LABEL)/(100*k)
print(f'Train data have lebel normal : {(list(EVERY_Fold_label_train_download[0]).count(0))/train_fraction:.2f}% and have lebel abnormal : {(list(EVERY_Fold_label_train_download[0]).count(1))/train_fraction:.2f}%')
print(f'Validation data have lebel normal : {(list(EVERY_Fold_label_val_download[0]).count(0))/val_fraction:.2f}% and have lebel abnormal : {(list(EVERY_Fold_label_val_download[0]).count(1))/val_fraction:.2f}%')
print(f'Test data have lebel normal : {100*(list(Test_label).count(0))/len(Test_label):.2f}% and have lebel abnormal : {100*(list(Test_label).count(1))/len(Test_label):.2f}%')

Train data have lebel normal : 49.58% and have lebel abnormal : 50.42%
Validation data have lebel normal : 49.94% and have lebel abnormal : 50.06%
Test data have lebel normal : 50.20% and have lebel abnormal : 49.80%


In [10]:
np.save(f'{DATA_type}/Train_data.npy', EVERY_Fold_data_train_download)
np.save(f'{DATA_type}/Train_label.npy', EVERY_Fold_label_train_download)
np.save(f'{DATA_type}/Val_data.npy', EVERY_Fold_data_val_download)
np.save(f'{DATA_type}/Val_label.npy', EVERY_Fold_label_val_download)
np.save(f'{DATA_type}/Test_data.npy', Test_data)
np.save(f'{DATA_type}/Test_label.npy', Test_label)